In [ ]:
import glob
import os
import subprocess
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy import view, plot_network
from mintpy.objects import gnss, timeseries, ifgramStack
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, writefile, utils as ut
from scipy import signal
from skimage.transform import rescale
from urllib.request import urlretrieve
from matplotlib.backends.backend_pdf import PdfPages


In [ ]:
def get_corners(atr):
    """Get corners coordinate."""
    length = int(atr['LENGTH'])
    width = int(atr['WIDTH'])
    W = float(atr['X_FIRST'])
    N = float(atr['Y_FIRST'])
    lon_step = float(atr['X_STEP'])
    lat_step = float(atr['Y_STEP'])
    S = N + lat_step * length
    E = W + lon_step * width

    return S, N, W, E, width, length
def rescale_data(data, mask, meta, ref_meta):
    """rescale matrix into a different resolution"""

    # calc scaling factor
    scale = (float(meta['Y_STEP']) / float(ref_meta['Y_STEP']),
             float(meta['X_STEP']) / float(ref_meta['X_STEP']))
    
    data = np.nan_to_num(data,nan=0.0);
    # scale
    if data.ndim > 2:
        data_out = np.stack([rescale(data[i], scale, mode='symmetric', preserve_range=True, order=0) for i in range(data.shape[0])]);
        len, wid = data_out.shape[1], data_out.shape[2]
    else:
        data_out = rescale(data, scale,mode='symmetric', preserve_range=True, order=0);
        len, wid = data_out.shape[0], data_out.shape[1]
    mask_out = rescale(mask, scale,mode='symmetric', preserve_range=True, order=0);
    data_out = np.where(mask_out, data_out, np.nan)
    # update metadata
    meta['Y_STEP'] = ref_meta['Y_STEP']
    meta['X_STEP'] = ref_meta['X_STEP']
    meta['LENGTH'], meta['WIDTH'] = len, wid

    return data_out, mask_out, meta

In [ ]:
# better start with an ARIA product on the north most (west most) part
ts_file_list= [r'C:\Users\bvarugu\Documents\Stockton\Asc\mintpy\corrected_ts_TRE_GNSS.h5',
                r'C:\Users\bvarugu\Documents\Concord\Asc\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Asc\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Losangeles1\Asc\mintpy\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Losangeles2\Asc\mintpy\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Southbay\geo\corrected_ts_TRE_GNSS.h5']
mask_file_list= [r'C:\Users\bvarugu\Documents\Stockton\Asc\mintpy\maskTempCoh_0.9.h5',
                r'C:\Users\bvarugu\Documents\Concord\Asc\maskTempCoh_0.9.h5',
               r'C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Asc\maskTempCoh_0.9.h5',
               r'C:\Users\bvarugu\Documents\Losangeles1\Asc\mintpy\maskTempCoh_0.9.h5',
               r'C:\Users\bvarugu\Documents\Losangeles2\Asc\mintpy\maskTempCoh_0.9.h5',
               r'C:\Users\bvarugu\Documents\Southbay\geo\maskTempCoh_0.9.h5']
geom_file_list= [r'C:\Users\bvarugu\Documents\Stockton\Asc\mintpy\geometryGeo.h5',
                r'C:\Users\bvarugu\Documents\Concord\Asc\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Asc\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Losangeles1\Asc\mintpy\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Losangeles2\Asc\mintpy\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Southbay\geo\geo_geometryRadar.h5']

In [ ]:
def get_common_dates(track1_date_list,track2_date_list):
    track1_datestamps = [dt.datetime.strptime(date, "%Y%m%d") for date in track1_date_list];
    track2_datestamps = [dt.datetime.strptime(date, "%Y%m%d") for date in track2_date_list];
    common_dates= np.full(len(track1_date_list), np.nan).tolist()
    for i, date in enumerate(track1_datestamps):
        closest_date = min(track2_datestamps, key=lambda b:abs(date-b));
        diff = date-closest_date;#diff = diff.astype(int)
        if abs(diff) < dt.timedelta(10):
            common_dates[i]= closest_date.strftime("%Y%m%d");
    return common_dates
disp_data_list = []; mask_list = []; atr_list= [];
desired_track_number = 0; #first track in the list
for i in range(len(ts_file_list)):
    ts_file = ts_file_list[i];
    mask = readfile.read(mask_file_list[i])[0]; mask_list.append(mask);   
    los_inc_angle = readfile.read(geom_file_list[i], datasetName='incidenceAngle')[0];
    if i==desired_track_number:
        track1_date_list = timeseries(ts_file).get_date_list();print(track1_date_list)
        ref_data, ref_atr = readfile.read(ts_file);
        ref_date = track1_date_list[0]; # first date of the track
        insar_up = (ref_data)/(np.cos(np.deg2rad(los_inc_angle)));
        disp_data_list.append(insar_up);atr_list.append(ref_atr);
    else:
        dates = timeseries(ts_file).get_date_list();
        common_dates = get_common_dates(track1_date_list,dates);
        #print(common_dates)
        atr = timeseries(ts_file).get_metadata();
        if atr['REF_DATE'] != common_dates[0]:
            print('Reference date not set to first date. Setting it ....');
            iargs = [ts_file, '--ref-date', str(common_dates[0])]
            import mintpy.cli.reference_date
            mintpy.cli.reference_date.main(iargs)
        data = np.zeros([len(ref_data), int(atr['LENGTH']), int(atr['WIDTH'])], dtype=np.float32) * np.nan;
        for j, common_date in enumerate(common_dates):
            if isinstance(common_date, str) and common_date:  # Ensure it's a valid string
                disp = readfile.read(ts_file, datasetName=common_date)[0];
                data[j, :, :] = (disp)/(np.cos(np.deg2rad(los_inc_angle)));
        disp_data_list.append(data);atr_list.append(atr);
            

In [ ]:
# define the grid boundaries
S,N,W,E = 33,39,-123,-115.5
print("S N W E :",S,N,W,E);
try:
    if ref_atr['X_STEP'] != str(0.000833333):
        raise ValueError("Set ref atr to an ARIA track")
    else:
        print("Ref step is:", ref_atr['X_STEP'])
except ValueError as e:
    print("Error:", e)
# ref_atr['X_STEP'] = 0.000833333; #this is equivalent ot 100m
# ref_atr['Y_STEP'] = -0.000833333;
lon_step = float(ref_atr['X_STEP'])
lat_step = float(ref_atr['Y_STEP'])
width  = int(np.ceil((E - W) / lon_step))
length = int(np.ceil((S - N) / lat_step))
print('Combined matrix shape is:',length,width)    

In [ ]:
S_bounds=[];N_bounds=[];W_bounds=[];E_bounds=[];lengths=[];widths=[];
#S_bounds.append(S1);N_bounds.append(N1);W_bounds.append(W1);E_bounds.append(E1);lengths.append(length1);widths.append(width1)
for i,atr in enumerate(atr_list):
    ratio_x = abs((float(ref_atr['X_STEP']) - float(atr['X_STEP'])) / float(ref_atr['X_STEP']))
    ratio_y = abs((float(ref_atr['Y_STEP']) - float(atr['Y_STEP'])) / float(ref_atr['Y_STEP']))
    # print(atr['FILE_PATH'],ratio_x,ratio_y)
    if any(ratio > 1e-3 for ratio in [ratio_x, ratio_y]):
        print('rescaling the matrix {} into the same spatial resolution as the reference grid'.format(i))
        disp_data_list[i], mask_list[i], atr = rescale_data(disp_data_list[i], mask_list[i], meta=atr, ref_meta=ref_atr);
        atr_list[i] = atr

    # input spatial extents
    print('grabbing corners of input matrices')
    S_bound, N_bound, W_bound, E_bound, width_bound, length_bound = get_corners(atr);
    S_bounds.append(S_bound);N_bounds.append(N_bound);W_bounds.append(W_bound);E_bounds.append(E_bound);lengths.append(length_bound);widths.append(width_bound);


In [ ]:
print('estimate difference in the overlapping area')
lon_seq = np.linspace(W, W + width  * lon_step, width, endpoint=False);
lat_seq = np.linspace(N, N + length * lat_step, length, endpoint=False);
lons,lats= np.meshgrid(lon_seq,lat_seq);
num_dates = len(track1_date_list);
num_tracks= len(atr_list)
merged_ts = np.zeros([num_dates,length, width]) * np.nan;
for d in range(num_dates):
    mat11 = np.zeros([num_tracks,length, width]) * np.nan;
    for i in range(num_tracks):
        x1, y1 = np.argmin(np.square(lon_seq - W_bounds[i])), np.argmin(np.square(lat_seq - N_bounds[i]));
        disp_data = disp_data_list[i][d]; mask = mask_list[i];
        disp_data[mask==0] = np.nan;
        mat11[i, y1:y1+lengths[i], x1:x1+widths[i]] = disp_data;
    merged_ts[d,:,:] = np.nanmean(mat11,axis=0);

    

In [ ]:
atr = dict()
for key, value in ref_atr.items():
    atr[key] = value
atr['FILE_TYPE'] = 'timeseries'
atr['WIDTH']  = str(width)
atr['LENGTH'] = str(length)
atr['X_STEP'] = str(lon_step)
atr['Y_STEP'] = str(lat_step)
atr['X_FIRST'] = str(W)
atr['Y_FIRST'] = str(N)
print(f'update LENGTH/WIDTH: {length}/{width}')
print(f'update Y/X_FIRST: {N}/{W}')

# update REF_Y/X
coord = ut.coordinate(atr)
ref_y, ref_x = coord.geo2radar(float(atr['REF_LAT']), float(atr['REF_LON']))[:2]
atr['REF_Y'], atr['REF_X'] = ref_y, ref_x
print(f'update REF_Y/X: {ref_y}/{ref_x}')

# delete SUBSET_Y/XMIN/MAX
for key in ['SUBSET_XMIN', 'SUBSET_XMAX', 'SUBSET_YMIN', 'SUBSET_YMAX']:
    if key in atr.keys():
        atr.pop(key)
        print(f'remove {key}')

In [ ]:
file_outname = r'C:\Users\bvarugu\Documents\merged_asc_GNSS_corrected_timeseries_TRE_GNSS.h5'
final_dates = np.array(track1_date_list, dtype=np.bytes_);
ts_dict = {
    "date"       : final_dates,
    "timeseries" : merged_ts,
}
writefile.write(ts_dict,file_outname,metadata=atr);

In [ ]:
# southbay_atr = atr_list[-1].copy();
# south_bay_data = disp_data_list[-1].copy();
# southbay_mask = mask_list[-1].copy();
# southbay_data_out, mask_out, atr_out = rescale_data(south_bay_data, southbay_mask, meta=southbay_atr, ref_meta=ref_atr);

In [ ]:
#plt.imshow(disp_data_list[0][45],cmap='jet',vmin=-0.1,vmax=0.1,interpolation='nearest')